### Imports

In [ ]:
# Datetime, for logging
from datetime import datetime
from datetime import date

# Pandas, for reading excel files
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
# Mame sure the package openpyxl is installed to write to excel files

# System stuff, for authentication and path stuff
from __future__ import print_function
import pickle
import os.path
import io
from io import BytesIO

# structs to help organize events
from collections import namedtuple

# Google stuff
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

### Some constant variable definitions:

In [ ]:
# Some important variables that may change over time
teamFolderName = "HSV Fellows Team Folder" # What is the exact name of the shared drive folder

attendanceFolderName = "Attendance Sheets" # And where do we store attendance records

outputSheetName = f"CompiledList_{datetime.now()}.xlsx"

logFileName = f"{datetime.now()}_output_log.txt" # Where we will store all the output of the program

debugMode = True # If this is enabled, all of the output will also be printed to the screen (as well as the log file)

credentialsFileName = "ncsu_credentials.json" # The credentials file that you download from Google

pickleFileName = "token.pickle" # Where the credentials will be stored

attendanceFolderSearchDepth = 2 # How many levels of directories we should look to find the attendance

downloadFolder = 'downloads' # Our downloads folder (make sure there isn't a trailing '/')

Event = namedtuple("Event", "name date villagePoints hosts id")

# This is both the format that the excel files should use, and the one we use to compare dates internally
dateFormat = "%m/%d/%Y"

# The following two variables define the window in which we are looking for events
lowerBoundDate = date(2020, 1, 1)
upperBoundDate = date(2020, 1, 31)

### Some simple methods that will help us with various things

In [ ]:
# This will help us log the output of the program
def log(string, alsoPrint):
    with open(logFileName, "a") as logFileBuffer:
        logFileBuffer.write(f"[{datetime.now()}]:  {string}\n")
        
    if alsoPrint:
        print(f"[{datetime.now()}]:  {string}")

### Method to setup authentication

In [ ]:
def setupAuthentication():
    
    creds = None
    
    #SCOPES = ['https://www.googleapis.com/auth/drive.file']
    SCOPES = ['https://www.googleapis.com/auth/drive']
    
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(pickleFileName):
        with open(pickleFileName, 'rb') as token:
            creds = pickle.load(token)
            
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        log(f"Didn't find credentials from {pickleFileName}", debugMode)
        
        # Either way set up the credentials
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
            log(f"Refreshing credentials via Google", debugMode)
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentialsFileName, SCOPES)
            creds = flow.run_local_server(port=0)
            log(f"Refreshing credentials via secrets file {credentialsFileName}", debugMode)
            
        # Save the credentials for the next run
        with open(pickleFileName, 'wb') as token:
            pickle.dump(creds, token)
    else:
        log(f"Found credentials from {pickleFileName}", debugMode)
        
    return creds

### Method to find the Team folder ID

In [ ]:
def findTeamFolderID(service):
    
    # This query string specifies that we are looking only for files that fit the mimetype in '', which means folders
    queryString = "mimeType = 'application/vnd.google-apps.folder'"
    # List all of the files that match our query
    results = service.files().list(q=queryString, pageSize=300, fields="nextPageToken, files(name, id)").execute()
    # Now grab those bad boys so we can mess around with them
    items = results.get('files', [])

    # We are going eventually single out the Fellow Team Folder, and we will store its GDrive ID here
    # This could also be found by looking at the long hexadecimal string in the url of the drive folder
    # but we want to find it automatically
    teamFolderID = None
    
    if not items:
        # Hopefully we don't find nothing, but it we do we wanna stop
        log('No files found in drive', debugMode)
        return
    else:
        # Otherwise we wanna print out that we found the folder and save the ID
        for item in items:
            if item['name'] == teamFolderName:
                log(u'Found team folder: ({0}, {1})'.format(item['name'], item['id']), debugMode)
                teamFolderID = item['id']

    if teamFolderID == None:
        # We didn't find the team folder, which is bad
        log('Team folder not found, try checking your team drive folder name and make sure it is exactly correct.', debugMode)
        return
    
    return teamFolderID

### Method to list files in a folder recursively (used in the next two methods)

In [ ]:
def recursiveListFilesInFolder(rootFolderID, depth, mimeType):

    queryString = "'" + rootFolderID + "' in parents and not trashed"
    folderSearch = service.files().list(q=queryString, pageSize=100, fields="nextPageToken, files(name, id, mimeType)").execute()
    files = folderSearch.get('files', [])

    allFiles = []

    for file in files:
        if file['mimeType'] == mimeType:
            allFiles.append(file)
        if depth != 0 and file['mimeType'] == 'application/vnd.google-apps.folder':
            allFiles = allFiles + recursiveListFilesInFolder(file['id'], depth - 1, mimeType)

    return allFiles


### Method to find the Attendance folder ID

In [ ]:
def findAttendanceFolderID(teamFolderID, service):
        
    # Recursively look for folders in the team folder, so we can search for our attendance one
    files = recursiveListFilesInFolder(teamFolderID, attendanceFolderSearchDepth, 'application/vnd.google-apps.folder')
    
    attendanceFolderID = None
    # Look for our attendance folder
    if not files:
        log('No files found', debugMode)
    else:
        for file in files:
            if file['name'] == attendanceFolderName:
                log(u'Found attendance folder: ({0}, {1})'.format(file['name'], file['id']), debugMode)
                attendanceFolderID = file['id']
                
    if attendanceFolderID == None:
        # We didn't find the folder where attendance sheets are kept
        log(f'Specified attendance folder not found, make sure that the folder name is exactly correct. You can also try increasing the search depth (current={attendanceFolderSearchDepth})', debugMode)
        return
    
    return attendanceFolderID

### Method to list all sheets in the attendance folder

In [ ]:
def getSheetIDInFolder(attendanceFolderID):
    # A depth of -1 means it will search until there are no more directories left ie. infinite depth
    # Let's just hope there aren't any symlinks in this folder :/
    spreadsheets = recursiveListFilesInFolder(attendanceFolderID, -1, 'application/vnd.google-apps.spreadsheet')
    
    if len(spreadsheets) == 0:
        log(f"No spreadsheets found in attendance folder with ID {attendanceFolderID}", debugMode)
        return
    
    spreadsheetIDNames = [[spreadsheets[i]["id"], spreadsheets[i]["name"]] for i in range(len(spreadsheets))]
    
    for s in spreadsheets:
        log(f"Found sheet {s}", debugMode)
        
    return spreadsheetIDNames
    

### Method to download all of the sheets

In [ ]:
def downloadSheets(sheetIDList, service):
    
    fileList = ['' for i in range(len(sheetIDList))]
    
    log(f"Beginning download for {len(sheetIDList)} files", debugMode)
    
    i = 0
    for sheetID in sheetIDList:
        
        log(f"Downloading file with ID {sheetID[0]} ({sheetID[1]})...", debugMode)
        
        request = service.files().export_media(fileId=sheetID[0],
                                                 mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        
        with open(f"{downloadFolder}/{sheetID[0]}.xlsx", 'wb') as file:
            file.write(fh.getvalue())
        fh.close()
        
        fileList[i] = [f"{downloadFolder}/{sheetID[0]}.xlsx", sheetID[0], sheetID[1]]
        i = i + 1
        
        log(f"Download completed for file with ID {sheetID[0]} ({sheetID[1]}): saved to {downloadFolder}/{sheetID[0]}.xlsx", debugMode)
        
    log(f"Download for {len(sheetIDList)} files complete", debugMode)

    return fileList

### The method to parse excel sheets for event/student info

In [ ]:
def parseSheet(sheetFile):
    
    # The test file we will be working on
    #sheetName = "Career Studio Takeover.xlsx"

    #try:
        # Load in the excel file using pandas
    file = pd.read_excel(sheetFile[0], parse_dates=True)
    #except:
    #    log(f'File {sheetName} not found!', debugMode)
    #    return None
    
    #print(file.columns)

    # We can't expect that everyone will format the name exactly as it should be,
    # so we want to identify what each header is actually called
    # Below we have a list of words that should show up in one way or another for each category
    keywords = {'Student ID':["id", 'student', 'ncsu'],
                'Event Name':['event', 'program', 'name'],
                'Student Name':['name', 'first'],
                'Village Points':['village', 'points', 'vp'],
                'Hosts':['host', 'fellow', 'leader'],
                'Date':['date', 'day', 'event', 'program']}
    actualKeywords = {}

    # These are characters that may show up that we don't want to influence the identification process
    arbitraryChars = [':', '(', ')', '[', ']', '{', '}']

    for header in file.columns:
        # Make it lowercase
        currHeader = header.lower()

        # We want to ignore columns that don't have headers
        if 'Unnamed' in header:
            continue

        # Remove semicolons
        for c in arbitraryChars:
            currHeader = currHeader.replace(c, '')

        #print(currHeader)

        qualifierMatches = {}
        i = 0
        for keyword, qualifiers in keywords.items():
            qualifierMatches[keyword] = 0
            for q in qualifiers:
                if q in currHeader:
                    qualifierMatches[keyword] = qualifierMatches[keyword] + 1

            # This is kinda an obscure way of weighting things, but it seems to work for now
            qualifierMatches[keyword] = float(qualifierMatches[keyword]) / (float(len(currHeader.split())) * float(len(keywords[keyword])))
            i = i + 1

        # Now we want to find which index of qualifierMatches has the highest value, which will become the actualKeyword
        key = max(qualifierMatches, key=qualifierMatches.get)
        #print(key)
        actualKeywords[key] = header

        #print(qualifierMatches)

        
    #print(actualKeywords)

    # To rule out now properly formatted files, we should make sure that every value in our dictionary is unique
    # ie. no two columns were assigned to the same header string or vice versa
    # We do this by flipping our dictionary and making sure the length stays the same, since keys must be unique
    flippedKeywords = {}
    for k, v in actualKeywords.items():
        flippedKeywords[v] = k
    
    #print(len(flippedKeywords))
    #print(len(keywords))
    
    if not len(flippedKeywords) == len(keywords):
        log(f"ERROR: Headers in file {sheetFile[0]} ({sheetFile[2]}) are not formatted properly, ignoring this file!", debugMode)
        return None
    
    # We now know where all of our information is, so we should start grabbing it
    
    
    villagePoints = None
    # We now know where all of our information is, so we should start grabbing it
    possibleVillagePoints = file[actualKeywords['Village Points']].dropna().values.tolist()
    #print(possibleVillagePoints)
    
    if isinstance(possibleVillagePoints, list):
        if len(possibleVillagePoints) == 1:
            villagePoints = possibleVillagePoints[0]
            log(f"Found value for village points: {villagePoints} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
        elif len(possibleVillagePoints) > 1:
            log(f"Warning: more than one populated cell under header {actualKeywords['VillagePoints']}; attempting to take first value", debugMode)
            villagePoints = possibleVillagePoints[0]
            log(f"Found value for village points: {villagePoints} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
        else:
            log(f"ERROR: Village point value for {sheetFile[0]} ({sheetFile[2]}) not specified, ignoring this file!", debugMode)
            return None
    else:
        villagePoints = possibleVillagePoints
        log(f"Found value for village points: {villagePoints} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)

    eventDate = None
    # Find our possible dates
    
    # We have to have an extra try except here since if there is no date, the .dt will get mad
    try:
        possibleDates = file[actualKeywords['Date']].dropna().dt.strftime('%m/%d/%Y').values.tolist()
    except:
        log(f"ERROR: Event date for {sheetFile[0]} ({sheetFile[2]}) not specified, ignoring this file!", debugMode)
        return None
    #print(possibleDates)
        
    if isinstance(possibleDates, list):
        if len(possibleDates) == 1:
            eventDate = possibleDates[0]
            log(f"Found value for event date: {eventDate} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
        elif len(possibleDates) > 1:
            log(f"Warning: more than one populated cell under header {actualKeywords['Date']}; attempting to take first value", debugMode)
            eventDate = possibleDates[0]
            log(f"Found value for event date: {eventDate} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)

        else:
            log(f"ERROR: Event date for {sheetFile[0]} ({sheetFile[2]}) not specified, ignoring this file!", debugMode)
            return None
    else:
        eventDate = possibleDates
        log(f"Found value for event date: {eventDate} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
    
    
    hosts = None
    possibleHosts = file[actualKeywords['Hosts']].dropna().values.tolist()
    #print(possibleHosts)
    
    if isinstance(possibleHosts, list):
        # We can have more than one host, so this is alright to have more than one cell populated
        if len(possibleHosts) >= 1:
            hosts = possibleHosts[0]
            log(f"Found value for hosts: {hosts} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
        else:
            log(f"ERROR: Hosts for {sheetFile[0]} ({sheetFile[2]}) not specified, ignoring this file!", debugMode)
            return None
    else:
        hosts = possibleHosts
        log(f"Found value for hosts: {hosts} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)

    eventName = None
    possibleNames = file[actualKeywords['Event Name']].dropna().values.tolist()
    #print(possibleNames)
    
    if isinstance(possibleNames, list):
        if len(possibleNames) == 1:
            eventName = possibleNames[0]
            log(f"Found value for event name: {eventName} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
        elif len(possibleNames) > 1:
            log(f"Warning: more than one populated cell under header {actualKeywords['Event Name']}; attempting to take first value", debugMode)
            eventName = possibleNames[0]
            log(f"Found value for event date: {eventName} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)
        else:
            log(f"ERROR: Event name for {sheetFile[0]} ({sheetFile[2]}) not specified, ignoring this file!", debugMode)
            return None
    else:
        eventName = possibleNames
        log(f"Found value for event name: {eventName} in file {sheetFile[0]} ({sheetFile[2]})", debugMode)

    # Create a struct for the event
    event = Event(eventName, eventDate, villagePoints, hosts, sheetFile[1])
    
    #print([villagePoints, eventDate, hosts, eventName])

    students = file[actualKeywords['Student ID']].dropna().values.tolist()
    #print(students)
    
    # Now that we have all of the information about the event, we should return all of the students who atended and the information about the event
    return [event, students]

### The method to save the student files to a sheet

In [ ]:
def saveStudentsToFile(studentList):
    
    # Pandas is kinda weird in that every column we write has to have the same length
    # This means that our column with generation data, since it alwaus has the same number
    # of columns, is the minimum number of columns (ie students) that we can run this program with
    # I know it seems stupid, but I don't think it's very important to fix right now
    if len(studentList) < 7:
        log(f"ERROR: Attempting to run program with less than 7 students, see method saveStudentsToFile for why this is invalid! Exiting...", debugMode)
        return
    
    # We will use this method to have the name of each event hyperlink to that attendance sheet
    def createLink(eventName, sheetID):
        return f"=HYPERLINK(\"https://docs.google.com/spreadsheets/d/{sheetID}\", \"{eventName}\")"
            
    # We've got to set up our data in columns so that it's easy to write
    # The student ID column is easy, we pretty much already have it
    studentIDList = list(studentList.keys())
        
    # Now iterate over every event for each student to get the rest of the info
    totalVillagePoints = [0 for i in range(len(studentIDList))]
    # Since excel and google sheets can only handle one hyperlink per cell, we will just have the
    # rest of the columns as space to hold one event per cell
    # Because of this, our list of events is actually 2D
    studentListOfEvents = [[] for i in range(len(studentIDList))]
    i = 0
    for ID, events in studentList.items():
        # We have to handle the case where there is only one event (and so python turns lists with only one element into just that element)
        if not isinstance(events, list):
            totalVillagePoints[i] = events.villagePoints
            studentListOfEvents[i] = [createLink(events.name, events.id)]
        else:
            for e in events:
                totalVillagePoints[i] = totalVillagePoints[i] + e.villagePoints
                studentListOfEvents[i] = studentListOfEvents[i] + [createLink(e.name, e.id)]
            
        i = i + 1
        
        
    #print(studentListOfEvents)
    
    # We now have to do a bit of reorganzing so that our events are easily writable
    maxColumns = max([len(elem) if isinstance(elem, list) else 1 for elem in studentListOfEvents])
    rectangularEventLists = [["" for j in range(len(studentIDList))] for i in range(maxColumns)]
    
    for i in range(len(studentIDList)):
        if isinstance(studentListOfEvents[i], list):
            for j in range(len(studentListOfEvents[i])):
                rectangularEventLists[j][i] = studentListOfEvents[i][j]
        else:
            rectangularEventLists[j][i] = studentListOfEvents[i]
    
    writableEventColumns = {}
    
    if isinstance(rectangularEventLists[0], list) and len(rectangularEventLists[0]) > 1:
    
        writableEventColumns["Events (1)"] = rectangularEventLists[0]

        for i in range(maxColumns - 1):
            writableEventColumns[f"{i+2}"] = rectangularEventLists[:][i+1]

    else:
        writableEventColumns["Events"] = rectangularEventLists[0][:]
    
    # The first column will just be full of generation info
    # We have to make a lot of empty cells since every column has to have the same length
    generationInfo = ["" for i in range(len(studentIDList))]
    generationInfo[0] = "Begin date:"
    generationInfo[1] = f"{lowerBoundDate}"
    generationInfo[3] = "End date:"
    generationInfo[4] = f"{upperBoundDate}"
    generationInfo[6] = "Execution date:"
    generationInfo[7] = f"{datetime.now()}"

    allColumns = {"Generation Info":generationInfo, "Student ID:":studentIDList, "Village Points:":totalVillagePoints}
    allColumns.update(writableEventColumns)
    
    log(f"Data manipulation for writing to file complete, beginning writing process!", debugMode)
    
    #print(allColumns)
    
    #for j, k in allColumns.items():
    #    print(len(k))
    
    #print(f"{len(generationInfo)} {len(studentIDList)} {len(totalVillagePoints)}")
    
    # Now we actually write to the file
    fileWriter = ExcelWriter(outputSheetName)
    log(f"Writing database to {outputSheetName}", debugMode)
    
    #data = pd.DataFrame([generationInfo, studentIDList, totalVillagePoints])
    data = pd.DataFrame(allColumns)
    data.to_excel(fileWriter,"Sheet1",index=False)
    fileWriter.save()
    
    log(f"Writing process completed successfully!", debugMode)

### The method to upload the sheet to drive

In [ ]:
def uploadSheetAndLogToDrive(service, attendanceFolderID):
    
    log(f"Beginning upload of compiled list into parent folder with ID {attendanceFolderID}", debugMode)
    
    # Our file is of type excel spreadsheet, but we can easily convert it to a google sheet format here
    sheetMetadata = {"name":outputSheetName,
                   "mimeType":"application/vnd.google-apps.spreadsheet",
                   "parents":[attendanceFolderID]}
    
    # Specifiy the original mimetype, not what it will end up as here (see above)
    sheetMedia = MediaFileUpload(outputSheetName,
                           mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
                           resumable=True)
    
    sheetUploadedFile = service.files().create(body=sheetMetadata,
                                 media_body=sheetMedia,
                                 fields='id').execute()
    
    log(f"Uploaded compiled sheet to specified attendance folder completed successfully!", debugMode)
    log(f"Uploaded file has ID {sheetUploadedFile.get('id')}", debugMode)
    
    log(f"Beginning upload of log file into parent folder with ID {attendanceFolderID}", debugMode)
    
    # Not as complex as the above expression
    logMetadata = {"name":logFileName,
                      "parents":[attendanceFolderID]}
    
    log(f"Log must end here, since file is being uploaded, for final status updates, see local copy of log.", debugMode)
    log(f"See you space cowboy...", debugMode)
    
    # Specifiy the original mimetype, not what it will end up as here (see above)
    logMedia = MediaFileUpload(logFileName,
                           mimetype="text/plain",
                           resumable=True)
    
    logUploadedFile = service.files().create(body=logMetadata,
                                 media_body=logMedia,
                                 fields='id').execute()
    
    log(f"Uploaded log file to specified attendance folder completed successfully!", debugMode)
    log(f"Uploaded file has ID {logUploadedFile.get('id')}", debugMode)


### The method to delete all of the downloaded files

In [ ]:
def cleanUpDownloadFolder():
    
    files = os.listdir(downloadFolder)
    
    log(f'Removing {len(files)} files from {downloadFolder}', debugMode)
    
    for f in files:
        os.remove(f'{downloadFolder}/{f}')
        log(f'Removed file {downloadFolder}/{f})', debugMode)
    
    log(f'Removed {len(files)} files', debugMode)
    
    return

## Finally, put it all together

In [ ]:
# First, we want to setup our credentials
creds = setupAuthentication()

# Create an object to interact with Google Drive
service = build('drive', 'v3', credentials=creds)

# Fetch the team folder ID
teamFolderID = findTeamFolderID(service)

# Find the attendance folder
attendanceFolderID = findAttendanceFolderID(teamFolderID, service)

# List all of the sheets inside the attendance folder
sheetIDList = getSheetIDInFolder(attendanceFolderID)

# Download all of the sheets and get a list of file names
fileList = downloadSheets(sheetIDList, service)

studentList = {}

for file in fileList:
    results = parseSheet(file)
    
    if results == None:
        continue
    
    # Make sure that the date is within the range we want
    # Not that we also have to cast from a datetime object to a date object
    actualDate = datetime.date(datetime.strptime(results[0].date, dateFormat))
    if actualDate < lowerBoundDate or actualDate > upperBoundDate:
        log(f"WARNING: Event {results[0].name} has date outside of specified range, ignoring this file!", debugMode)
        continue
        
    for s in results[1]:
        try:
            studentList[int(s)] = [studentList[s]] + [results[0]]
        except:
            studentList[int(s)] = results[0]

# Save everything in a nice format
saveStudentsToFile(studentList)

# And remove all of the files we downloaded
cleanUpDownloadFolder()

# Upload to our attendance folder
uploadSheetAndLogToDrive(service, attendanceFolderID)

# This won't actually show up in the drive since the file is uploaded before this, but :/
log("Execution completed", debugMode)